# 네이버 자주 본 뉴스 크롤링

1) 수집내용

    1) 많이본뉴스–섹션별(정치~IT/과학)Top5기사제목,신문사,뷰 
    2) 해당 기사별 기사 내용, 리액션 (좋아요 ~ 후속기사 원해요)
    
2) 수집방법(택1)

    1) [기본] Requests , BeautifulSoup, Selenium
    2) [심화] Requests, BeautifulSoup (+ 멀티프로세싱)
    
3) 수집범위 및 저장

    1) 2019년7월21일~2020년8월20일(동작가능,실제구동x)
    2) 하나의 파일로 저장 (방식 자유)
    3) Ex)총6섹션*Top5*365일=10950rows

기본적으로 크롤링에 필요한 패키지들을 import한다.

In [124]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from IPython.display import Image
import os
from pathlib import Path
import glob

import datetime

## 날짜 String 생성

과제의 요구사항이 1년치 네이버 랭킹뉴스를 크롤링 하는 것이기 때문에, 날짜 string 생성하는 것은 중요한 과정이다. 아래의 과정을 통해 날짜 string을 생성한다.

In [125]:
start_date = datetime.datetime(2019, 7, 21, 3, 22, 32)
start_date.strftime("%Y%m%d %Y-%m-%d %H:%M:%S ") #strftime 활용 예시

'20190721 2019-07-21 03:22:32 '

In [126]:
start_date1 = datetime.datetime(2019, 7, 21, 3, 22, 32)
start_date1.strftime("%Y%m%d %Y-%m-%d 시간 표시외 %H:%M:%S 아무거나 넣어도 됩니다.".encode('unicode-escape').decode()).encode().decode('unicode-escape') #strftime 활용 예시

'20190721 2019-07-21 시간 표시외 03:22:32 아무거나 넣어도 됩니다.'

처음과 같이 코드를 짜면 날짜를 만들 수 있다. 하지만 그 안에 한글을 섞기 위해서는 decode와 incode를 반복해야하는 번거로움이 있다.

In [127]:
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [128]:
date_string_list = get_date_string(start_date, 366)

In [129]:
date_string_list[0], date_string_list[-1] # 생성 완료

('20190721', '20200720')

위와 같이 함수를 만들어서 1년치 날짜가 모두 리스트 형식으로 나오게끔 받아준다. 1년은 365일이지만 366일로 만든 이유는 올해가 윤년이기 때문이다. string_list의 처음과 끝을 확인함으로서 list가 잘 만들어졌는지 확인한다.

## Part 1. Request를 이용한 크롤링

request를 통해 우선 기본적인 뉴스 데이터를 긁어온다. 우선 url을 받아주고 request로 우리가 사용할 페이지를 설정한다.  

그리고 각각 신문제목, 기사링크, 신문사명, view 수를 구해준다. 과정은 거의 일치한다. 우선 빈 리스트를 만들어준다. 이후 soup.select를 통해 내가 원하는 정보가 담긴 경로를 설정한다. 이렇게 select를 통해 긁어온 데이터는 리스트형태로 받아오나 이 중 우리가 원하는 정보는 text에 담겨있다. for문을 돌려 각 요소의 text를 빈 리스트에 넣어준다. 이 과정에서 약간의 차이는 발생한다. 링크의 경우, a tag 안의 href에 들어있다. 따라서 [href]로 인덱싱해, 그 결과값을 도출해야한다. 신문사명의 경우, 그대로 받아오면 원하는 정보만 나오는 것이 아니라 부가적인 것들이 같이 나온다. 이를 위해 약간의 전처리를 진행해야 한다.   

마지막으로 다시 한 번 for문을 돌려서 지금까지 구한 리스트의 각 요소별로 묶어주고 이를 result 리스트에 넣어준다. 그리고 이를 return하면 크롤링 함수는 완성된다.

In [130]:
def get_top_news(date):
    """
    해당 날짜의 자주 본 뉴스 25개에 대한 정보를 반환하는 함수입니다.
    """
    url = f"https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&date={date}" 
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    result = []
    
    # soup.select를 잘 활용하여
    # 1. 신문 제목
    title_list = []
    title = soup.select('ol > li > dl a')
    for i in range(len(title)):
        title_list.append(title[i].text)
       
    # 2. 기사 링크 (a tag의 href 속성) 
    link_list = []
    link = soup.select('ol > li > dl a')
    for i in range(len(link)):
        link_list.append(link[i]['href'])
    
    # 3. 신문사명
    news_list = []
    news = soup.select('ol > li > dl span:nth-child(2)')
    for i in range(len(news)):
        news_list.append(news[i].text.strip())
        
    # 4. View
    view_list = []
    view = soup.select('ol > li > dl i')
    for i in range(len(view)):
        view_list.append(view[i].text)
    
    for i in range(len(title_list)):
        list_ = [title_list[i], link_list[i], news_list[i], view_list[i]]
        result.append(list_)
    
    
    
    return result

In [131]:
TEST_DATE_RANGE = 20 # 1년치 전체를 하는 것이 아닌 앞 20일치만 테스트 해보기 위함 (개인적으로 바꿔도 무방)

news_data = []
for date in date_string_list[:TEST_DATE_RANGE]:
     news_data.extend(get_top_news(date)) # 결과로 나온 25개를 이어 붙임

In [132]:
len(news_data)

600

In [133]:
news_data[:5]

[['[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=032&aid=0002952683&date=20190721&type=1&rankingSectionId=100&rankingSeq=1',
  '경향신문',
  '201,039'],
 ['조국 폭풍페북, 日주장 정면반박…"친일파" 표현은 野 반발',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=008&aid=0004251344&date=20190721&type=1&rankingSectionId=100&rankingSeq=2',
  '머니투데이',
  '185,396'],
 ['조국, 연일 對日 \'항전\' 주문…"겁먹고 쫄지말자…싸워 이겨…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0010969325&date=20190721&type=1&rankingSectionId=100&rankingSeq=3',
  '연합뉴스',
  '130,198'],
 ['[김순덕의 도발]복수를 하려면 아일랜드처럼!',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=020&aid=0003230442&date=20190721&type=1&rankingSectionId=100&rankingSeq=4',
  '동아일보',
  '120,897'],
 ['조국, 또 페북에 反日 선전전..."文정부, 서희·이순신 역할…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003462170&date=20190721&type=1&rankingSectio

함수의 결과는 위와 같다. 새 list인 news_data는 리스트를 요소로 받고 있으며 요소는 뉴스의 제목, 링크, 신문사, 조회 수 순으로 나오고 있다. 이 결과물을 좀 더 보기 쉽게 만들기 위해 데이터 프레임으로 변환한다.

In [134]:
# 결과물을 데이터 프레임으로 변환 및 Column Name 부여
df_top_news = pd.DataFrame(news_data, columns=["title", "url", "press", "views"])

In [135]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,조선일보,"119,463"
...,...,...,...,...
595,"갤노트10 만져본 미국인들 ""실물 보고싶어 일부러 찾아와""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,서울경제,"173,703"
596,"""유튜브 뛰어드니""…월급 295만→536만원 '껑충' vs 소득 '…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,뉴스1,"134,014"
597,"[SNS 세상] ""공감이 최고의 위로"" 암 투병기 연재하는 뷰티…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,"111,786"
598,네이버·카카오 2분기 성적표 '희비'…하반기 '페이전쟁' 격…,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,뉴스1,"77,278"


데이터 프레임은 잘 만들어졌다. 하지만 여기서 주의해서 봐야할 점은 url 부분이다. 저런 형태의 url은 해당 사이트로 들어갈 수 없고 오류를 발생시킨다. 따라서 앞에 http://news.naver.com 붙여주는 과정이 필요하다. 아래와 같이 전처리를 하면 깔끔하게 해결된다.

In [136]:
df_top_news.url = df_top_news['url'].apply(lambda x : "{}{}".format('http://news.naver.com',x))

In [137]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",http://news.naver.com/main/ranking/read.nhn?mi...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",http://news.naver.com/main/ranking/read.nhn?mi...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",http://news.naver.com/main/ranking/read.nhn?mi...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,http://news.naver.com/main/ranking/read.nhn?mi...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",http://news.naver.com/main/ranking/read.nhn?mi...,조선일보,"119,463"
...,...,...,...,...
595,"갤노트10 만져본 미국인들 ""실물 보고싶어 일부러 찾아와""",http://news.naver.com/main/ranking/read.nhn?mi...,서울경제,"173,703"
596,"""유튜브 뛰어드니""…월급 295만→536만원 '껑충' vs 소득 '…",http://news.naver.com/main/ranking/read.nhn?mi...,뉴스1,"134,014"
597,"[SNS 세상] ""공감이 최고의 위로"" 암 투병기 연재하는 뷰티…",http://news.naver.com/main/ranking/read.nhn?mi...,연합뉴스,"111,786"
598,네이버·카카오 2분기 성적표 '희비'…하반기 '페이전쟁' 격…,http://news.naver.com/main/ranking/read.nhn?mi...,뉴스1,"77,278"


## Part 2. Selenium을 이용한 크롤링

위에서는 request만을 사용하여 크롤링을 진행했다. 지금부터는 selenium을 같이 사용해 크롤링을 진행하려고 한다. 우선 driver_path 및 driver를 실행해보자. 과정은 아래와 같다. driver의 경우 노트북 파일과 같은 곳에 있어야 한다. 아래 코드를 실행하면 실제 드라이버가 잘 작동함을 알 수 있다.

In [138]:
driver_path = '/Users/jj950/chromedriver'

In [139]:
driver = webdriver.Chrome(driver_path) # Chrome driver 실행

In [140]:
df_top_news.url[:10]

0    http://news.naver.com/main/ranking/read.nhn?mi...
1    http://news.naver.com/main/ranking/read.nhn?mi...
2    http://news.naver.com/main/ranking/read.nhn?mi...
3    http://news.naver.com/main/ranking/read.nhn?mi...
4    http://news.naver.com/main/ranking/read.nhn?mi...
5    http://news.naver.com/main/ranking/read.nhn?mi...
6    http://news.naver.com/main/ranking/read.nhn?mi...
7    http://news.naver.com/main/ranking/read.nhn?mi...
8    http://news.naver.com/main/ranking/read.nhn?mi...
9    http://news.naver.com/main/ranking/read.nhn?mi...
Name: url, dtype: object

이제 selenium을 통해 기사 내용과 리액션 투표 수를 크롤링하려고 한다. 우선 예시로 10개의 url에 대해서만 진행해보자.  

우선 options를 통해 driver를 headless로 만들어둔다. 그렇지 않으면 크롤링을 진행하는 과정에서 무수히 많은 창들이 생성된다. 기본적인 크롤링 세팅은 위와 동일하다. driver_path를 설정하고, driver를 띄우는 과정이다. 그리고 time.sleep을 둠으로써 시간적 차이를 만들어준다. 그렇지 않으면 화면에 정보가 나타나기전에 긁어오는 현상이 벌어져 정보를 제대로 크롤링하지 못하는 상황이 벌어졌다.  

selenium은 beautifulsoup보다 비교적 함수가 간단하다. find_element_by_css_selector를 사용해 해당 위치의 class를 입력하면 정보를 긁어올 수 있다.  

하지만 content와 like는 차이가 있는데 content의 경우, 전체 html에서 해당 class가 저것 하나기 때문에 element를 사용하고, 받아오는 결과 역시 단일로 나와 바로 text를 사용할 수 있다. 또한 그 이후의 전처리들 역시 바로 적용 가능하다. 그러나 like의 경우, 하나가 아니기 때문에 elements를 사용해야한다. 또한 list의 형태이기 때문에 데이터 전처리를 위해선 for문 등이 더 필요하다.  

마지막으로 받아온 likes를 [5:]로 인덱싱 해준다. 실제로 like부분을 따로 돌려보게 되면 총 10개의 데이터가 크롤링 되고, 그 중 앞의 5개는 공란이다. 이를 해결하기 위해서 아래와 같이 인덱싱 해준다.  

그리고 마지막으로 만들어둔 데이터 프레임에 새롭게 열을 생성해주면 완성이다.

In [141]:
NEWS_TEST_RANGE = 10
for idx, news_url in enumerate(df_top_news.url[:NEWS_TEST_RANGE]):
    # 드라이버 내에서 해당 URL로 이동
    from selenium.webdriver.chrome.options import Options
    import time
    
    options = Options()
    options.headless = True
    driver_path = '/Users/jj950/chromedriver'
    driver = webdriver.Chrome(driver_path, options = options)
    driver.get(news_url)
    time.sleep(1)
    
    ## BeautifulSoup 혹은 driver.find_element[s]_by_css_selector 을 이용하여 정보 파싱
    # +기사 내용
    content = driver.find_element_by_css_selector('div._article_body_contents').text.replace('\n','').strip()
    
    # +5가지 리액션 (좋아요, 훈훈해요, 슬퍼요, 화나요, 후속기사원해요) 투표 수
    
    like = driver.find_elements_by_css_selector('span.u_likeit_list_count._count')
    likes = []
    for i in range(len(like)):
        likes.append(like[i].text)
    
    likes = likes[5:]
        
    # 예시로 content라는 변수에 기사 내용을 담고 Column "content"에 해당 내용 저장
    df_top_news.loc[idx,"content"] = content
    df_top_news.loc[idx, "like"] = likes[0]
    df_top_news.loc[idx, "warm"] = likes[1]
    df_top_news.loc[idx, "sad"] = likes[2]
    df_top_news.loc[idx, "upset"] = likes[3]
    df_top_news.loc[idx, "want"] = likes[4]

위 함수의 결과는 아래와 같다. 제목, url, 신문사, 조회수, 기사내용, 리액션 투표수까지 깔끔하게 정리된 것을 볼 수 있다.

In [142]:
df_top_news.head(5)

,title,url,press,views,content,like,warm,sad,upset,want
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",http://news.naver.com/main/ranking/read.nhn?mi...,경향신문,"201,039",중 3 때 오빠와 장관상이어 고3 때도 ‘장함모’ 활동으로 자원봉사대회 금상“대체로...,131,16,17,"2,605",121
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",http://news.naver.com/main/ranking/read.nhn?mi...,머니투데이,"185,396","[머니투데이 김성휘 ,백지수 기자] [[the300]징용판결 해설 글 ""대통령 법률...","1,710",32,13,"6,058",48
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",http://news.naver.com/main/ranking/read.nhn?mi...,연합뉴스,"130,198","""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""""文정부 매도 정치인·...","1,799",19,13,"9,120",30
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,http://news.naver.com/main/ranking/read.nhn?mi...,동아일보,"120,897","친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적이라면, 문재인 정부...","3,017",19,16,623,37
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",http://news.naver.com/main/ranking/read.nhn?mi...,조선일보,"119,463","""문재인 정부, 국익 수호 위해 '서희' '이순신' 역할 함께 수행""""법적·외교적 ...",374,13,14,"11,468",38


> 본 데이터 프레임은 예시로, 리액션 데이터를 담지 않았습니다. **과제에는 리액션 데이터가 반드시 담겨져있어야합니다.**

# 데이터 저장

> 파일 형태로 크롤링한 데이터를 저장

위에서 test를 마쳤으므로 이제 본격적으로 1년치 기사의 크롤링을 진행하려고 한다. 과정은 아래와 같다. 우선 date_string_list를 만들어주고, 이를 get_top_news에 넣어준다. 만들어진 news_data를 데이터프레임으로 만들어주고, 간단한 전처리를 한다. 이 후 긁어온 url을 바탕으로 기사와 리액션 투표수를 긁어온다. 마지막으로 이를 csv 파일로 저장해준다.  

실제 돌려보고 싶으나, 노트북 사양이 암울하므로 여기서 마치겠습니다...

In [ ]:
date_string_list = get_date_string(start_date, 366)

news_data = []
for date in date_string_list:
     news_data.extend(get_top_news(date))

In [ ]:
df_top_news = pd.DataFrame(news_data, columns=["title", "url", "press", "views"])
df_top_news.url = df_top_news['url'].apply(lambda x : "{}{}".format('http://news.naver.com',x))

In [ ]:
for idx, news_url in enumerate(df_top_news.url):
    # 드라이버 내에서 해당 URL로 이동
    from selenium.webdriver.chrome.options import Options
    import time
    
    options = Options()
    options.headless = True
    driver_path = '/Users/jj950/chromedriver'
    driver = webdriver.Chrome(driver_path, options = options)
    driver.get(news_url)
    time.sleep(1)
    
    ## BeautifulSoup 혹은 driver.find_element[s]_by_css_selector 을 이용하여 정보 파싱
    # +기사 내용
    content = driver.find_element_by_css_selector('div._article_body_contents').text.replace('\n','').strip()
    
    # +5가지 리액션 (좋아요, 훈훈해요, 슬퍼요, 화나요, 후속기사원해요) 투표 수
    
    like = driver.find_elements_by_css_selector('span.u_likeit_list_count._count')
    likes = []
    for i in range(len(like)):
        likes.append(like[i].text)
    
    likes = likes[5:]
        
    # 예시로 content라는 변수에 기사 내용을 담고 Column "content"에 해당 내용 저장
    df_top_news.loc[idx,"content"] = content
    df_top_news.loc[idx, "like"] = likes[0]
    df_top_news.loc[idx, "warm"] = likes[1]
    df_top_news.loc[idx, "sad"] = likes[2]
    df_top_news.loc[idx, "upset"] = likes[3]
    df_top_news.loc[idx, "want"] = likes[4]

In [ ]:
df_top_news.to_csv('top_news_crawling.csv', index = False)